In [1]:
import pandas as pd
from sklearn.linear_model import Lasso

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
train = pd.read_csv("30_Training Dataset_V2/training_data.csv")
test = pd.read_csv("30_Public Dataset_Public Sumission Template_v2/public_dataset.csv")
Y_train = train[['單價']]
print(test.shape)

(5876, 21)


In [3]:
con_data = pd.concat([train, test], axis=0)
con_data.shape

(17627, 22)

In [4]:
con_data = con_data.drop(["ID","單價","使用分區",'備註'], axis=1)
con_data = pd.get_dummies(con_data, columns=['縣市','鄉鎮市區','路名','主要用途','主要建材','建物型態'])
con_data.head(5)

,土地面積,移轉層次,總樓層數,屋齡,建物面積,車位面積,車位個數,橫坐標,縱坐標,主建物面積,...,主要建材_其他,主要建材_加強磚造,主要建材_磚造,主要建材_鋼筋混凝土加強磚造,主要建材_鋼筋混凝土造,主要建材_鋼骨造,建物型態_住宅大樓(11層含以上有電梯),建物型態_公寓(5樓含以下無電梯),建物型態_華廈(10層含以下有電梯),建物型態_透天厝
0,-0.256716,11,11,32.583333,-0.174154,-0.819326,0.0,305266,2768378,0.393926,...,0,0,0,0,1,0,1,0,0,0
1,0.100134,7,12,24.166667,0.314204,-0.819326,0.0,300677,2767990,-0.316131,...,0,0,0,0,1,0,1,0,0,0
2,0.181921,10,15,6.166667,0.423366,0.161624,1.0,184815,2504666,-0.098871,...,1,0,0,0,0,0,1,0,0,0
3,0.085594,9,14,8.833333,0.164249,0.524653,1.0,296653,2772355,-0.071147,...,0,0,0,0,1,0,1,0,0,0
4,-0.938116,41,43,11.000000,0.985839,0.532377,1.0,297377,2768472,0.791954,...,0,0,0,0,0,1,1,0,0,0


In [5]:
X_train = con_data[:11751]
X_test = con_data[11751:]

In [6]:
from sklearn.linear_model import Lasso,LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

In [7]:
Y_train = Y_train.values.ravel()

### Optimal Alpha by using k-fold
measured by metrics such as mean squared error or R-squared score

In [9]:
# alphas = np.random.uniform(0,100,500)

# # 不需要執行標準化
# steps = [
#     # ('scaler',StandardScaler()),
#     ('lasso',LassoCV(alphas = alphas, cv = 10, random_state = 91))
# ]

# pipeline = Pipeline(steps)
# pipeline.fit(X_train, Y_train)

Pipeline(steps=[('lasso',
                 LassoCV(alphas=array([31.85331213, 31.78306559, 74.12106112, 46.60970619, 22.91126229,
       57.73325932, 47.72346789, 50.67594347, 19.90168505,  5.12439343,
       73.54749919, 18.53780953, 29.00069431, 72.4531913 , 70.87764306,
        9.00552161,  2.36232362, 97.49217883, 31.39016644, 53.38712655,
       56.32494506, 73.56274714, 56.09240828, 88.32698237, 50.83716871,
       27.2807...
       32.21613491,  6.03196661, 70.49441111,  8.593431  , 37.7522946 ,
       57.9941768 , 52.64426726, 50.63066185, 16.7136775 , 67.19770309,
       22.04706047, 19.11676874, 37.12842115, 83.05988174, 16.61719772,
       67.0912001 , 49.43326809, 37.47168891, 53.47394793, 24.01893868,
       59.13236998, 66.83283311, 43.55250233, 41.26793473, 58.1550725 ,
       36.4076936 , 97.89535207, 90.07600764, 28.88704364, 39.38228891]),
                         cv=10, random_state=91))])

In [45]:
# best_alpha = pipeline.named_steps['lasso'].alpha_
# best_alpha

0.05358914495500189

In [10]:
best_alpha = 0.00099
model = Lasso(alpha = best_alpha, max_iter = 50000)
model = Lasso()
model.fit(X_train, Y_train)

Lasso()

In [11]:
pred = model.predict(X_test)
pred

array([2.17842699, 2.25920205, 2.26505733, ..., 2.31545739, 2.37182924,
       2.27723923])

In [13]:
result = pd.read_csv("30_Public Dataset_Public Sumission Template_v2/public_submission_template.csv")
result

,ID,predicted_price
0,PU-1,0.0
1,PU-2,0.0
2,PU-3,0.0
3,PU-4,0.0
4,PU-5,0.0
...,...,...
5871,PU-5872,0.0
5872,PU-5873,0.0
5873,PU-5874,0.0
5874,PU-5875,0.0


In [14]:
result['predicted_price'] = pred
result.to_csv("./result.csv", index=False)

# 增加外部資料

In [4]:
ext_data = pd.read_csv("國中小及設施數.csv")
ext_data

,ID,縣市,鄉鎮市區,路名,土地面積,使用分區,移轉層次,總樓層數,主要用途,主要建材,...,單價,lon,lat,geometry,viewpoint,facility,國中(高),國中(中),國中(下),國小(明星)
0,TR-2937,台北市,中正區,忠孝西路一段,-1.425204,None,7,15,商業用,鋼筋混凝土造,...,3.620538,121.518647,25.046344,POINT (121.51864667160082 25.046343978622435),0,292,0,0,0,0
1,TR-2073,台北市,信義區,信義路五段,0.730054,None,5,11,住家用,鋼筋混凝土造,...,5.330396,121.563510,25.033090,POINT (121.56350954469465 25.033089905398718),5,220,0,0,0,0
2,TR-213,台北市,中正區,濟南路一段,-1.034947,None,2,8,住家用,鋼筋混凝土造,...,5.470932,121.521811,25.042460,POINT (121.52181144998664 25.042459846908113),0,217,0,0,0,0
3,TR-5370,台北市,中正區,徐州路,1.914236,None,5,5,住家用,鋼筋混凝土造,...,3.878188,121.523250,25.040423,POINT (121.52324967176007 25.040423468773227),0,214,0,0,0,0
4,TR-10523,台北市,中正區,延平南路,-1.425180,None,12,14,住家用,鋼筋混凝土造,...,4.159260,121.511115,25.043960,POINT (121.5111148404516 25.043959580920827),7,211,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11746,TR-7292,新北市,汐止區,東勢街,0.817344,其他,7,10,其他,鋼筋混凝土造,...,2.753898,121.632900,25.041974,POINT (121.6328995877812 25.04197400113705),0,0,0,0,0,0
11747,TR-2631,高雄市,鳳山區,北昌二街,-0.005950,None,12,15,集合住宅,鋼筋混凝土造,...,1.184577,120.371905,22.646011,POINT (120.3719053959096 22.646011003047192),0,0,0,0,0,0
11748,TR-11294,高雄市,鳳山區,北昌五街,-0.229222,None,8,15,集合住宅,鋼筋混凝土造,...,1.048725,120.374303,22.642977,POINT (120.37430282549076 22.64297701272843),0,0,0,0,0,0
11749,TR-4228,桃園市,八德區,廣興五街,0.047525,None,2,13,住家用,鋼筋混凝土造,...,1.254845,121.279708,24.933310,POINT (121.27970782444278 24.9333103609338),0,0,0,0,0,0


In [5]:
test = pd.read_csv("30_Public Dataset_Public Sumission Template_v2/public_dataset.csv")
test

,ID,縣市,鄉鎮市區,路名,土地面積,使用分區,移轉層次,總樓層數,主要用途,主要建材,...,屋齡,建物面積,車位面積,車位個數,橫坐標,縱坐標,備註,主建物面積,陽台面積,附屬建物面積
0,PU-1,新北市,樹林區,大有路,2.039378,None,3,7,住家用,鋼筋混凝土造,...,13.916667,0.472202,0.146176,1.0,288745,2760266,NaN,0.849664,0.146161,-0.438452
1,PU-2,新北市,三重區,力行路一段,-0.077786,None,5,5,住家用,鋼筋混凝土造,...,40.833333,-0.546455,-0.819326,0.0,298751,2773492,NaN,-0.004102,0.064257,-0.438452
2,PU-3,新北市,板橋區,民生路三段,-0.555766,None,2,20,集合住宅,鋼筋混凝土造,...,1.333333,0.124032,-0.167419,1.0,297000,2769106,NaN,-0.285862,-0.775260,1.323042
3,PU-4,新北市,泰山區,明志路二段,-0.744838,None,6,16,住家用,鋼筋混凝土造,...,27.500000,-0.027072,0.115280,1.0,293076,2771160,NaN,0.027582,0.316794,-0.139532
4,PU-5,台北市,信義區,忠孝東路五段,-0.610923,None,3,7,住家用,鋼筋混凝土造,...,40.500000,-0.863600,-0.819326,0.0,308103,2770220,NaN,-0.883328,-0.560262,-0.075477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5871,PU-5872,台南市,中西區,中華西路二段,0.560113,None,5,5,集合住宅,鋼筋混凝土造,...,0.833333,-0.788335,-0.016028,1.0,166751,2545157,NaN,-0.867769,-1.078987,-0.438452
5872,PU-5873,台中市,北區,育樂街,-1.020142,None,5,22,集合住宅,鋼筋混凝土造,...,3.416667,-0.910137,0.799628,1.0,218814,2672567,NaN,-1.107095,-0.754784,-0.438452
5873,PU-5874,新北市,新店區,安康路二段,-0.662691,None,10,12,住家用,鋼筋混凝土造,...,12.583333,-0.459699,0.004054,1.0,301609,2761359,NaN,-0.474267,-0.131972,-0.438452
5874,PU-5875,台北市,信義區,忠孝東路五段,0.513752,None,2,5,住家用,鋼筋混凝土造,...,36.250000,0.357294,-0.819326,0.0,308833,2770624,NaN,0.926327,1.243333,-0.316749
